# EDA, the Polars Way

What is [polars](https://docs.pola.rs/)?

- Library for manipulation with tabular data*, based on arrow
- Contender to [pandas](https://pandas.pydata.org/) (and many more similar tools)
- Since 2020, started by Ritchie Vink

### Why polars?

- Performance (rust)
- Clean(er) API
- SQL-like concepts (?)
- Lazy evaluation & query optimization
- Cool kid on the block

### Why not polars?

- ? Less stable (version 1.26 in April 2025)
- ? Less functionality 
- Less known
- Sometimes lengthy code
- Copilot tends to suggest pandas code ;-)

## Let's start

```shell
jupyter lab
```

or

```shell
uv run jupyter lab
```

or Visual Studio Code or PyCharm if you prefer those.

Open "exercises.ipynb"

In [1]:
# Uncomment and run this if you're using deepnote
# !pip install -r requirements.txt

In [4]:
# Most basic import
import polars as pl

# Other useful imports
from datetime import date, datetime
import polars.selectors as cs
import hvplot.polars  # To support older way of plotting


%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


## Fundamental data structures

See https://docs.pola.rs/user-guide/concepts/data-structures/

### DataFrame

- a "table"?
- a "spreadsheet" table?
- a "dict of columns"?

In [5]:
# Load some data
un = pl.read_csv("data/un_basic.csv", try_parse_dates=True)
un


iso3,country,population,area,admission_date,region,subregion
str,str,i64,f64,date,str,str
"""AFG""","""Afghanistan""",41128771,652860.0,1946-11-19,"""Asia""","""Southern Asia"""
"""ALB""","""Albania""",2777689,28750.0,1955-12-14,"""Europe""","""Southern Europe"""
"""DZA""","""Algeria""",44903225,2.381741e6,1962-10-08,"""Africa""","""Northern Africa"""
"""AND""","""Andorra""",79824,470.0,1993-07-28,"""Europe""","""Southern Europe"""
"""AGO""","""Angola""",35588987,1.2467e6,1976-12-01,"""Africa""","""Sub-Saharan Africa"""
…,…,…,…,…,…,…
"""VEN""","""Venezuela""",28301696,912050.0,1945-11-15,"""Americas""","""Latin America and the Caribbea…"
"""VNM""","""Vietnam""",98186856,331340.0,1977-09-20,"""Asia""","""South-eastern Asia"""
"""YEM""","""Yemen""",33696614,527970.0,1947-09-30,"""Asia""","""Western Asia"""


In [ ]:
# What is it?
type(un)


### Series

- a "list"?
- a "column"?
- an "array of X"?

A bit of everything...

In [ ]:
# Select one column for a DataFrame
un["country"]


In [ ]:
type(un["country"])

### Closer look at the table

In [ ]:
un.shape

In [ ]:
un.columns

A random selection of rows using [`.sample`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.sample.html#polars.DataFrame.sample)

In [ ]:
un.sample(10)

Look at basic statistical properties using [`.describe`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.describe.html#polars.DataFrame.describe):

In [ ]:
un.describe()

Other useful methods to obtain a selection of rows:
- [`.head`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.head.html)
- [`.tail`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.tail.html)

### D(ata) types

- each column holds objects of the same type (unlike Python collections!)
- distinct from (but convertible to/from) Python classes
- the types are nullable => each value can be missing (difference to pandas)

In [ ]:
# List of all data types in a DataFrame
un.dtypes


In [ ]:
# More useful (dict)
{col: un[col].dtype for col in un.columns}


#### Common types

- Int8, Int16, Int32, Int64, UInt8, UInt16, UInt32, UInt64
- Float32, Float64
- Decimal
- Date, Datetime, Time
- String, Categorical, Enum
- Array, List, Struct
- Boolean
- Object, Null, Binary, Unknown

See https://docs.pola.rs/user-guide/concepts/data-types/overview/

In [ ]:
# Construct a Series from an object
pl.Series("city", ["Firenze", "Berlin", "Pittsburgh", "Prague"], dtype=pl.String)


In [ ]:
# Construct a DataFrame from dictionary of lists.

pl.DataFrame(
    {
        "event": ["PyCon Austria", "PyCon US", "PyCon Italia", "EuroPython"],
        "city": ["Eisenstadt", "Pittsburgh",  "Bologna", "Prague"],
        "country": ["Austria", "United States of America", "Italy", "Czechia"],
        "start_date": [
            date(2025, 4, 6),
            date(2024, 5, 14),
            date(2024, 5, 28),
            date(2025, 7, 14),
        ],
    }
)


In [ ]:
## Pandas window

import pandas as pd

pandas_series = pd.DataFrame(
    {"a": [1, 2, 3, 4, 5], "b": pd.Series([1, 2, 3, 4, 5], dtype="float64")}
)
pl.DataFrame(pandas_series)


### Indices

Polars, unlike pandas (with its complex index types, multi-indices etc.), does not have indices.
End of story.

**Exercise**: 
Load the list of cities from an external file and look at it.

In [1]:
# Exercise load-cities
# Load the data:
# - the file is called "data/worldcities.parquet"
# - find the appropriate loading method from https://docs.pola.rs/user-guide/io/
cities = pl....(...)
cities


SyntaxError: invalid syntax (2339476449.py, line 5)

## Basic plotting

Choose any library you want:

- [altair](https://altair-viz.github.io/)
- [plotly](https://plotly.com/python/)
- [matplotlib](https://matplotlib.org/)
- [seaborn](https://seaborn.pydata.org/)
- [hvplot](https://hvplot.holoviz.org/)
- ...

### "Built-in" plotting support

With altair installed, you can directly plot the dataframes:

```python
df.plot.line()
df.plot.bar()
df.plot.point()
...
```

However, `.hvplot` (provided `hvplot` is installed) offers (similar) access to older API that seem to achieve simple things in fewer lines of code:


In [ ]:
un.hvplot.scatter(
    x="area",
    y="population",
    logx=True,
    logy=True,
    color="region",
    title="Countries of the World",
    hover_cols=["country"],
)

Slightly more info here:
- https://hvplot.holoviz.org/user_guide/Pandas_API.html - pandas API for hvplot. It works almost the same with polars
- https://docs.pola.rs/py-polars/html/reference/dataframe/plot.html
- https://docs.pola.rs/user-guide/misc/visualization/
- https://altair-viz.github.io/ - altair documentation


**Exercise**: 

Draw the "poor man's map of the world", based on the "lat" and "lng" columns of the `cities` tabls.
Optionally: You can embellish it with any aesthetics you want.

In [2]:
# Exercise world-map
# Plot the cities
# - find the appropriate plotting method
# - use proper arguments for the call
cities = pl.read_csv("data/worldcities.parquet")
cities.hvplot....(
    x=...,
    y=...,
    ...
    legend=False,
    ...
)

SyntaxError: invalid syntax (3627296326.py, line 6)

## Sorting

- [.sort](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.sort.html)

In [ ]:
un.sort("admission_date")

Note: We receive a new DataFrame, as with all other manipulations. There is no "inplace" in polars. (unlike pandas, where this is only discouraged)

In [ ]:
un.sort("population", descending=True)

In [ ]:
un.sort("region", "subregion")

**Exercise:** Create a bar plot of 10 countries with the lowest population.

Hints:
- [`.sort`]()
- [`.head`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.head.html) 
- [`.plot.bar`](https://hvplot.holoviz.org/reference/tabular/bar.html) - docs from the hvplot pages
- Use `hover_cols` and `x` args to describe the plot properly (optional)

In [ ]:
# Exercise ten-smallest
sorted_population = ...
ten_smallest = ...

# Plot it
...

## Expressions & selection

To select one column, we can use the bracket notation:


In [ ]:
un["country"]

Something very similar can be achieved by using [`.select`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.select.html)

In [ ]:
# Find the difference
un.select("country")


And now we explicitly construct an expression ("choose the value in column country"):

In [ ]:
# Select a column
un.select(pl.col("country"))  # Or pl.col.country

Expression, representing column(s) in a dataframe:

In [ ]:
pl.col("country")

Expression describe operations (or trees of operations) that work on dataframes and produce one or more Series.

- they are evaluated only when needed
- they can be executed in chunks
- polars can optimize evaluation of the complex trees

See [User guide](https://docs.pola.rs/user-guide/concepts/expressions/) for more.

In [ ]:
# Literal value
un.select(pl.lit("country"))

Finally, we get to computing.

First, direct operations on series:

In [ ]:
un["population"] / un["area"]

In [ ]:
date.today() - un["admission_date"]

However, let's construct this as an expression:

In [ ]:
pl.col("population") / pl.col("area")

We can use expressions in `.select`:

In [ ]:
un.select(
    "country",
    "population",
    "area",
    (pl.col("population") / pl.col("area")).alias("density"),
).sort("density", descending=True)


In [ ]:
un.top_k(10, by=(pl.col("population") / pl.col("area")))

Btw. You can use SQL in polars!

See https://docs.pola.rs/api/python/stable/reference/sql/python_api.html.

In [ ]:
pl.sql("SELECT country, population, area, population / area AS density FROM un ORDER by density DESC LIMIT 10")

To extend a dataframe with new columns:

[`.with_columns`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.with_columns.html)

In [ ]:
un.with_columns(
    density=pl.col("population") / pl.col("area"),
)

Some other methods to change the columns:
- [`.drop`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.drop.html) to remove
- [`.rename`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.rename.html) to change column names
- [`.cast`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.cast.html) to change column data types

Note: You can pass an expression to `.sort` too.

**Exercise**: Add a column "membership_in_years",giving how long each country has been a member of the U.N.

- Use the difference from above
- Use [`polars.Expression.dt.total_days`](https://docs.pola.rs/py-polars/html/reference/expressions/api/polars.Expr.dt.total_days.html)
- Divide to get years
- (Optionally) If you want to export "complete" years, use interger division and `cast(pl.Int64)` to convert to integers

In [ ]:
# Exercise membership-years
un...(
    
)

## Filtering

When you want to select rows based on some criteria... Well, a short detour:

### Contexts

Every expression can only be executed within one of the following contexts:

1. Selection (`.select`, `.with_columns`) - we already saw this
2. Filtering (`.filter`)
3. Aggregation - we will see this

See https://docs.pola.rs/user-guide/concepts/contexts/.


Pass any boolean expression to the [`.filter`]() method

In [ ]:
un.filter(pl.col("population") > 1e9)

In [ ]:
un.filter(pl.col("country")=="Austria")

In [ ]:
un.filter(country="Czechia")

**Exercise:** Show how the share of electricity coming from different sources evolved over time in Austria (or from some other country).

In [ ]:
# Exercise energy-at
el_source = pl.read_csv(
    "data/our_world_in_data/electricity-source.csv", infer_schema_length=5000
)  # Note the infer_schema_length
el_source 
# el_source_at = ...
# el_source_at


**Exercise:** Create a linear (or stacked area) plot of the fractions

Hints:
- `.plot.area` is your friend
- you can supply multiple column names in the `y` argument
- An example is here: https://hvplot.holoviz.org/user_guide/Pandas_API.html#area-plot

In [ ]:
# Exercise energy-at
el_source_at.hvplot....(..., ...)


## Aggregations

When you want to find the summary statistics, you can use one of several Series/DataFrame functions, such as `.max`, `.mean`, `.median`, `.quantile`, ...

In [ ]:
un["population"].max()

In [ ]:
un.describe()

In [ ]:
un.max()  # Careful, it is per-column

In [ ]:
un.median()

It works on series too:

In [ ]:
un["area"].quantile(0.05)

**Exercise** Find all founding members of the U.N. 

Hints:
- find the admission date of the first member first
- you can call statistics methods on the Series directly
- you might want to find the proper minimizing function (min)

In [ ]:
# Exercise founding-members
first_date = ...
founding_members = ...
founding_members


### Grouped aggregations

[.group_by](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.group_by.html)

- it allows to select columns (or expression) to group by.
- in combination with a few selected functions or the `.agg`, it enters a new context for operations with the groups

In [ ]:
un.group_by("region")

In [ ]:
un.group_by("region").min()

In [ ]:
un.group_by("subregion").len()

In [ ]:
un.group_by("region", "subregion").len().sort("region", "subregion")

In [ ]:
un.group_by("subregion").sum()

The `.agg` method works similar to `.select`, only the expressions are evaluated only on the group and then combined together, together with the group keys.

In [ ]:
un.group_by("region", "subregion").agg(
    pl.sum("*"),
    total_population=pl.col("population").sum(),
    total_area=pl.col("area").sum(),
    num_countries=pl.col("area").count(),
    
).sort("region", "subregion")


In [ ]:
forest_area = pl.read_csv("data/our_world_in_data/forest-area-km.csv")
forest_area.filter(Code="AUT")

**Exercise:** Find the relative change of forestation for each country on the year range.

- group by an appropriate column
- find the proper aggregation functions (https://docs.pola.rs/user-guide/expressions/aggregation) - first, last might be your friends
- optionally exclude the infinite and NA values ([`.is_finite`](https://docs.pola.rs/py-polars/html/reference/expressions/api/polars.Expr.is_finite.html)) 

In [ ]:
# Exercise forest-change
first_and_last_forest_area = forest_area.group_by(...).agg(
    ...
)
relative_change = first_and_last_forest_area....
# Filter finite and sort
relative_change = relative_change....
relative_change

### Time aggregations

In [ ]:
weather = pl.read_parquet("data/vienna-meteostat.parquet")
weather


What about timezones? Let's forget about them for now... but it would deserve its own workshop. See https://docs.pola.rs/user-guide/transformations/time-series/timezones/

In [ ]:
weather.hvplot.line(y="temp")

In [ ]:
weather.hvplot.line(x="time", y="temp")

#### Intermezzo: Missing values

- Find them: [`.is_null`](https://docs.pola.rs/py-polars/html/reference/expressions/api/polars.Expr.is_null.html)
- Remove them: `.filter` is your friend, .drop_nulls
- Replace them: [`.fill_null`](https://docs.pola.rs/py-polars/html/reference/expressions/api/polars.Expr.fill_null.html)

More on that in [User Guide](https://docs.pola.rs/user-guide/expressions/missing-data/)

In [ ]:
weather.filter(pl.col("temp").is_null())

In [ ]:
weather.drop_nulls()

Don't confuse with NaN (pandas users!) and/or infinities:

In [ ]:
pl.Series([1, 0, None]) / pl.Series([0, 0, 0])

Back to time series

In [ ]:
# We can use the year column
yearly_mean = weather.group_by(
    pl.col("time").dt.year().alias("year"), maintain_order=True
).agg(avg_temp=pl.col("temp").drop_nans().mean())
yearly_mean


In [ ]:
yearly_mean.hvplot.line(x="year", y="avg_temp")

But groupping by calendar month? That might be trickier.

And hence the [`.group_by_dynamic`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.group_by_dynamic.html) method.

- it takes one column/expression to serve as a time index
- the data must be sorted in that column (either by previous `.sort` or [`.set_sorted`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.set_sorted.html))
- you can specify the aggregation period by a number of ways

In [ ]:
monthly = (
    weather
    .group_by_dynamic("time", every="1mo")
    .agg(avg_temp=pl.col("temp").mean() ) #.drop_nans().mean())
)
monthly.hvplot.line(x="time", y="avg_temp")


**Exercise:** What was the day with highest lowest temperature (probably meaning the hottest night) in Vienna since 1980?

Hints:
- Filter the new values only
- Group by an appropriate time period
- Find the minimum
- You might need to `.drop_nans` and `.drop_nulls` to work with reasonable values (at appropriate moment)
- Work with the minima to find the top value (or a few of them)

In [ ]:
# Exercise hottest-night
recent_weather = weather.filter(...)
min_daily_temperatures = recent_weather.group_by_dynamic(...).agg(...)
top_nights = min_daily_temperatures.....head(10)
top_nights

Similarly, you can group by a rolling window using [`.rolling`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.rolling.html), ...

**Exercise:** Can you find the highest rainfall within a 48-hour (rolling) window? (Not every 2 days)

In [ ]:
# Exercise highest-rainfall
weather....agg(...).top_k(10, by=...)

## Joining

We have data coming from two (or more) tables. We want to combine them, so we call the [`.join`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.join.html) method on of them and:

- which column(s) should match (`on`, `left_on`, `right_on` arguments)
- how to deal with missing values (`how`)
- how to deal with non-matching columns that are in both dataframes (`suffix`)
- whether to restrict the relationship somehow (`validate`)

More on that in the [User guide](https://docs.pola.rs/user-guide/transformations/joins/).

(Pandas note: the different, almost equivalent ways of joining in pandas are a big source of confusion)

In [ ]:
current_forest_area = forest_area.group_by("Code").last()
current_forest_area

In [ ]:
un

**Example:** Find the total forest area per region.

In [ ]:
current_forest_area.join(
    un, left_on="Code", right_on="iso3", how="inner"
).group_by("region").agg(pl.col("Forest area").sum(), pl.col("area").sum()).with_columns(
    forest_area_ratio=pl.col("Forest area") / pl.col("area") / 100 / 0.71
)


In [ ]:
cities

**Exercise** Find the number of cities over 1 million inhabitants per region / subregion. (Bonus: include the largest of those cities)

Hints:
- Join with the "un" table on appropriate columns
- Aggregate over appropriate columns and use [`.len`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.dataframe.group_by.GroupBy.len.html)

In [ ]:
# Exercise million-cities
million_cities = cities.filter(pl.col("population") > 1e6)
million_cities_with_country = million_cities.join(...)
million_cities_per_region = (
    million_cities_with_country.group_by(...)...
)
million_cities_per_region


## Wide / long table format

### Wide -> long

Sometimes we have a 2D table where the column names do not describe independent attributes but rather one condition under many circumstances, like in the following:

In [ ]:
wb_pop_wide = pl.read_csv("data/world_bank-population.csv")
wb_pop_wide


We can transform the column labels to one column and convert the table to "long format":

[.unpivot](https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.unpivot.html#polars.DataFrame.unpivot)

In [ ]:
wb_pop = (
    wb_pop_wide.unpivot(
        on=cs.numeric(),
        index=["Country Code", "Country Name"],
        variable_name="year",
        value_name="population",
    )
    .cast({"year": pl.Int64})
    .rename({"Country Code": "iso3", "Country Name": "country"})
)
wb_pop.sample(10)


In [ ]:
world_pop = wb_pop.filter(iso3="WLD").drop(
    "iso3", "country"
)  # .plot(x="year", y="population")
world_pop.hvplot.line(x="year", y="population")


In [ ]:
# Alternative
wb_pop_wide.filter(pl.col("Country Code") == "WLD").select(cs.numeric()).transpose(
    header_name="year", include_header=True
).rename({"column_0": "population"})


### Long -> wide (pivotting)

In the opposite of melting, we want to aggregate something over a pair of columns and create a "2D", pivot table. 

[.pivot]()

**Example:** Plot the daily pattern of temperatures in Vienna for each month of the year (since 2020).

In [ ]:
month_day_data = (
    weather
    .drop_nulls("temp")
    .filter(
        pl.col("temp").is_not_nan(),
        pl.col("time") >= date(2000, 1, 1),
    )
    .select("time", "temp")
    .with_columns(
        month=pl.col("time").dt.month(),
        month_name=pl.col("time").dt.strftime("%B").alias("month_name"),
        hour=pl.col("time").dt.hour(),
    )
).drop("month", "time")
month_day_data


In [ ]:
month_day_data.pivot(
    on="month_name",
    values="temp",
    index="hour",
    aggregate_function="mean",
)

In [ ]:
month_day_table = month_day_data.pivot(
    on="month_name",
    values="temp",
    index="hour",
    aggregate_function="mean",
)
month_day_table


In [ ]:
month_day_table.hvplot.line(x="hour")

**Exercise:** Find how the total forested area developed for different regions year by year and draw a stacked area plot.

Hints:
- Join `un` and `forecast_area` dataframes, using appropriate columns
- Pivot using the region as `column` and year as row `index`. Find the appropriate `value` column.
- Find the appropriate value for the `aggregate_function` argument.

Question: You should see something weird in the output - can you explain?

In [ ]:
# Exercise forest-region
forest_area_by_region = forest_area.join(un, ...).pivot(...)
forest_area_by_region

## Lazy operations

One of the best attributes of polars is its ability to optimise queries and perform them in chunks.

We can start lazy mode by
- lazy input functions ([`scan_csv`](https://docs.pola.rs/py-polars/html/reference/api/polars.scan_csv.html), [`scan_parquet`](https://docs.pola.rs/py-polars/html/reference/api/polars.scan_parquet.html), ...)
- calling [`.lazy`](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.lazy.html) on a dataframe

All operations on the lazy dataframes are added to the query.

The final results are obtained by a [`.collect`](https://docs.pola.rs/py-polars/html/reference/lazyframe/api/polars.LazyFrame.collect.html) or some other method call.

In [ ]:
lazy_cities = pl.read_parquet("data/worldcities.parquet")
lazy_cities

In [ ]:
lazy_mean_population = (
    lazy_cities.lazy().filter(pl.col("population") > 1e6)
    .group_by("country")
    .agg(pl.col("population").mean().alias("mean_population"))
    .sort("mean_population", descending=True)
)
lazy_mean_population


In [ ]:
print(lazy_mean_population.explain(optimized=True))

In [ ]:
pop = lazy_mean_population.collect()
pop

Find more on that in the [User Guide](https://docs.pola.rs/user-guide/lazy/). We do not have any larger-than-memory data file in our repo...

## Save our work!

Read the [user guide](https://docs.pola.rs/user-guide/io/).

In [ ]:
pop.write_excel("pop.xlsx")

In [6]:
p = un.hvplot.scatter(
    x="area",
    y="population",
    logx=True,
    logy=True,
    color="region",
    title="Countries of the World",
    hover_cols=["country"],
)

In [7]:
hvplot.save(p, "x.png")

RuntimeError: To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')